In [1]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

# IMPORTS
import numpy as np
import random
import itertools
import json
import os
# Machine Learning imports
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import PReLU, Input, LSTM, Flatten, Concatenate, Dense, Conv2D, TimeDistributed, MaxPooling2D, ReLU, Dropout, BatchNormalization, Activation, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Precision
import tensorflow_model_optimization
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
print("tensorflow version:", tf.__version__)
import keras
print("keras version:",keras.__version__)
import qkeras
from qkeras import QActivation, QDense, QConv2D, QBatchNormalization, QConv2DBatchnorm
from qkeras import quantized_relu, quantized_bits
from qkeras.utils import _add_supported_quantized_objects
from qkeras.autoqkeras.utils import print_qmodel_summary
print("qkeras version:",keras.__version__)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# Display and plotting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
from IPython.display import HTML, display
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import confusion_matrix
# Data management
import psutil
import h5py
# Memory management
import gc
# Notifications
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv
load_dotenv()
def send_email_notification(subject, content):
    sender_email = os.getenv('EMAIL_USER')
    receiver_email = "alexander.j.yue@gmail.com"
    password = os.getenv('EMAIL_PASS')

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    body = content
    message.attach(MIMEText(body, "plain"))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())

# Memory monitoring functions
def print_memory_usage():
    memory = psutil.virtual_memory()
    print(f"Total memory: {memory.total / (1024**3):.2f} GB")
    print(f"Available memory: {memory.available / (1024**3):.2f} GB")
    print(f"Used memory: {memory.used / (1024**3):.2f} GB")
    print(f"Memory usage percentage: {memory.percent}%")

def print_cpu_usage():
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"CPU Usage: {cpu_percent}%")

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

2024-07-15 11:45:30.079293: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 11:45:30.251997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 11:45:30.253854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 11:45:32.962797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


tensorflow version: 2.13.1
keras version: 2.13.1
qkeras version: 2.13.1


In [2]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

# Load the pixel cluster to transverse momentum dataset into the input_data and target_data
def load_combine_shuffle_data_optimized_hdf5():
    # Load the dataset from Kenny's computer
    with h5py.File('/fs/ddn/sdf/group/atlas/d/hjia625/Smart_Pixel/fl32_data_v3.hdf5', 'r') as h5f:
        combined_input = None
        combined_target = None

        for data_type in ['sig', 'bkg']:
            # Construct dataset names
            input_dataset_name = f'{data_type}_input'
            target_dataset_name = f'{data_type}_target'

            # Check if the dataset exists and load data sequentially
            if input_dataset_name in h5f and target_dataset_name in h5f:
                input_data = h5f[input_dataset_name][:].astype(np.float32)
                target_data = h5f[target_dataset_name][:].astype(np.float32)

                if combined_input is None:
                    combined_input = input_data
                    combined_target = target_data
                    # Free memory of the loaded data
                    del input_data, target_data
                    gc.collect()

                else:
                    print_memory_usage()
                    combined_input = np.vstack((combined_input, input_data))
                    combined_target = np.vstack((combined_target, target_data))
                    # Free memory of the loaded data
                    del input_data, target_data
                    gc.collect()

            else:
                print(f"Dataset {input_dataset_name} or {target_dataset_name} not found.")

        # Shuffling
        indices = np.arange(combined_input.shape[0])
        np.random.shuffle(indices)
        combined_input = combined_input[indices]
        combined_target = combined_target[indices]

        return combined_input, combined_target

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [3]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

def load_dataset():
    # Load dataset into memory
    input_data, target_data = load_combine_shuffle_data_optimized_hdf5()
    # Format the dataset into a 20x13x21 tensor (time, y, x)
    input_data = input_data.reshape(input_data.shape[0],20,13,21)
    return input_data, target_data

def process_dataset(input_data, target_data, hyperparams):
    NUM_TIME_SLICES = hyperparams["NUM_TIME_SLICES"]
    MODEL_TYPE = hyperparams["MODEL_TYPE"]
    TRAIN_PT_THRESHOLD = hyperparams["TRAIN_PT_THRESHOLD"]
    TEST_PT_THRESHOLD = hyperparams["TEST_PT_THRESHOLD"]
    INPUT_SCALING = hyperparams["INPUT_SCALING"]

    # Split 80% of data into training data, 10% for validation data and 10% for testing data
    input_train_data, input_temp, target_train_data, target_temp = \
    train_test_split(input_data, target_data, test_size=0.2, random_state=42)
    del input_data
    del target_data
    gc.collect()
    input_validate_data, input_test_data, target_validate_data, target_test_data = \
    train_test_split(input_temp, target_temp, test_size=0.5, random_state=42)
    del input_temp
    del target_temp
    gc.collect()

    # Save some data for displaying
    input_data_example = input_test_data[0:100,:]
    target_data_example = target_test_data[0:100,:]

    # Fit the scalers on the training data to it all scales the exact same
    if INPUT_SCALING == "Standard":
        input_scaler = StandardScaler()
        input_scaler.fit(input_train_data[:, :NUM_TIME_SLICES, :, :].reshape(-1,8*13))
        y0_scaler = StandardScaler()
        y0_scaler.fit(target_train_data[:,7].reshape(-1, 1))
    elif INPUT_SCALING == "MinMax":
        input_scaler = MinMaxScaler()
        input_scaler.fit(input_train_data[:, :NUM_TIME_SLICES, :, :].reshape(-1,8*13))
        y0_scaler = MinMaxScaler()
        y0_scaler.fit(target_train_data[:,7].reshape(-1, 1))
    elif INPUT_SCALING == "Log":
        _temp = "null" # no steps here 
    else: 
        raise ValueError(f"unsupported INPUT_SCALING {INPUT_SCALING}")

    # Process the data into input shape and labels for training
    def process_data(input_data, target_data, pt_threshold):
        if input_data.shape[1:] == (20, 13, 21) and target_data.shape[1:] == (13, ):

            # Truncate down to first time slices
            input_data = input_data[:, :NUM_TIME_SLICES, :, :]

            # sum over the x axis to turn the input data into a 2D NUM_TIME_SLICES x 13 tensor (time, y)
            input_data = np.sum(input_data, axis=3)

            # Encode the target data into one_hot encoding
            one_hot = np.zeros((target_data.shape[0], 3))
            # Assign 1 for p_t > pt_threshold in GeV, for low p_t put 1 in slot 2 for negative and a 1 in slot 3 for positive
            one_hot[np.abs(target_data[:, 8]) >= pt_threshold, 0] = 1
            one_hot[(np.abs(target_data[:, 8]) < pt_threshold) & (target_data[:, 8] > 0), 1] = 1
            one_hot[(np.abs(target_data[:, 8]) < pt_threshold) & (target_data[:, 8] < 0), 2] = 1

            # Flatten the input data
            input_data = input_data.reshape(-1,NUM_TIME_SLICES*13)

            # Get the y_0 data
            y0_data = target_data[:,7].reshape(-1, 1)

            # Normalize the input data according to scaling method
            if INPUT_SCALING == "Standard" or INPUT_SCALING == "MinMax":
                input_data = input_scaler.transform(input_data)
                y0_data = y0_scaler.transform(y0_data)
            elif INPUT_SCALING == "Log":
                # Replace all values < 1 with 1 so they log to 0
                input_data = np.where(np.abs(input_data) < 1.0, 1.0, input_data)
                # Apply logarithmic scaling
                input_data = np.log(np.abs(input_data)) * np.sign(input_data)
                # Min-max normalization (global)
                min_val = np.min(input_data)
                max_val = np.max(input_data)
                print(f"max of log of data is {max_val} and min is {min_val}")
                input_data = (input_data) / np.max([max_val,min_val])
            else: 
                raise ValueError(f"unsupported INPUT_SCALING {INPUT_SCALING}") 

            
            # Combine with input data
            if (MODEL_TYPE == "DNN"):
                # For DNN we concatenate in the y_0 data
                input_data_combined = np.hstack((input_data, y0_data))
            elif (MODEL_TYPE == "CNN"):
                # Reshape data into a matrix for the convolutions
                input_data = input_data.reshape(-1, NUM_TIME_SLICES, 13)
                # Package with the y_0 data to be added later
                input_data_combined = [input_data, y0_data]
            elif (MODEL_TYPE == "AE"):
                # Reshape data into a matrix for the convolutions
                input_data_combined = input_data.reshape(-1, NUM_TIME_SLICES, 13)
            else: 
                raise ValueError(f"unsupported MODEL_TYPE {MODEL_TYPE}") 
            
            return input_data_combined, one_hot
        else:
            raise ValueError("Wrong array shape!")

    # Apply data processing to our datasets
    input_train_data_combined, target_train_data_coded = process_data(input_train_data, target_train_data, TRAIN_PT_THRESHOLD)
    input_validate_data_combined, target_validate_data_coded = process_data(input_validate_data, target_validate_data, TRAIN_PT_THRESHOLD)
    input_test_data_combined, target_test_data_coded = process_data(input_test_data, target_test_data, TEST_PT_THRESHOLD)

    # Save some data for displaying
    if MODEL_TYPE == "DNN" or MODEL_TYPE == "AE":
        input_data_combined_example = input_test_data_combined[0:100,:]
        target_data_coded_example = target_test_data_coded[0:100,:]
    elif MODEL_TYPE == "CNN":
        input_data_combined_example = np.hstack((input_test_data_combined[0][0:100,:].reshape(100, -1), input_test_data_combined[1][0:100,:]))
        target_data_coded_example = target_test_data_coded[0:100,:]
    else:
        raise ValueError("Invalid model type")

    print_memory_usage()

    processed_dataset = {
        "input_train_data_combined": input_train_data_combined,
        "target_train_data_coded": target_train_data_coded,
        "input_validate_data_combined": input_validate_data_combined,
        "target_validate_data_coded": target_validate_data_coded,
        "input_test_data_combined": input_test_data_combined,
        "target_test_data_coded": target_test_data_coded,

        "input_data_example": input_data_example,
        "target_data_example": target_data_example,
        "input_data_combined_example": input_data_combined_example,
        "target_data_coded_example": target_data_coded_example,
    }

    return processed_dataset

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [13]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

# Defining the model
def qDNNmodel(hyperparams):
    NUM_TIME_SLICES = hyperparams["NUM_TIME_SLICES"]
    DNN_LAYERS = hyperparams["DNN_LAYERS"]
    WEIGHTS_BITS = hyperparams["WEIGHTS_BITS"]
    BIAS_BITS = hyperparams["BIAS_BITS"]
    ACTIVATION_BITS = hyperparams["ACTIVATION_BITS"]
    LEARNING_RATE = hyperparams["LEARNING_RATE"]
    
    y_timed_input = Input(shape=(NUM_TIME_SLICES*13 + 1,), name='y_timed_input')
    layer = y_timed_input
    
    for i, size in enumerate(DNN_LAYERS):
        layer = QDense(size, kernel_quantizer=quantized_bits(WEIGHTS_BITS), bias_quantizer=quantized_bits(BIAS_BITS), name=f'dense{i+1}')(layer)
        layer = BatchNormalization()(layer)
        layer = QActivation(quantized_relu(ACTIVATION_BITS))(layer)
        
    
    output = QDense(3, kernel_quantizer=quantized_bits(WEIGHTS_BITS), bias_quantizer=quantized_bits(BIAS_BITS), name='dense_output')(layer)
    output_softmax = Activation("softmax", name='output_softmax')(output)
   
    model = Model(inputs=y_timed_input, outputs=output_softmax)

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=[Precision()])

    return model


def qCNNmodel(hyperparams):
    NUM_TIME_SLICES = hyperparams["NUM_TIME_SLICES"]
    CONV_LAYER_DEPTHS = hyperparams["CONV_LAYER_DEPTHS"]
    CONV_LAYER_KERNELS = hyperparams["CONV_LAYER_KERNELS"]
    CONV_LAYER_STRIDES = hyperparams["CONV_LAYER_STRIDES"]
    MAX_POOLING_SIZE = hyperparams["MAX_POOLING_SIZE"]
    FLATTENED_LAYERS = hyperparams["FLATTENED_LAYERS"]
    WEIGHTS_BITS = hyperparams["WEIGHTS_BITS"]
    BIAS_BITS = hyperparams["BIAS_BITS"]
    INTEGER_BITS = hyperparams["INTEGER_BITS"]
    ACTIVATION_BITS = hyperparams["ACTIVATION_BITS"]
    LEARNING_RATE = hyperparams["LEARNING_RATE"]

    y_profile_input = Input(shape=(NUM_TIME_SLICES, 13, 1), name='y_profile_input')  # Adjust the shape based on your input
    layer = y_profile_input

    # Convolutional layers
    for i in range(len(CONV_LAYER_DEPTHS)):
        layer = QConv2D(
        CONV_LAYER_DEPTHS[i],
        kernel_size=CONV_LAYER_KERNELS[i],
        strides=CONV_LAYER_STRIDES,
        kernel_quantizer=quantized_bits(WEIGHTS_BITS,INTEGER_BITS,alpha=1.0),
        bias_quantizer=quantized_bits(BIAS_BITS,INTEGER_BITS,alpha=1.0),
        padding='same',
        use_bias=True,
        name=f'conv{i+1}'
        )(layer)
        layer = QActivation(quantized_relu(ACTIVATION_BITS), name=f'relu{i+1}')(layer)
        layer = MaxPooling2D(pool_size=MAX_POOLING_SIZE, name=f'maxpool{i+1}')(layer)

    # Flatten the output to feed into a dense layer
    layer = Flatten(name='flattened')(layer)

    # Flatten and concatenate with y0 input
    y0_input = Input(shape=(1,), name='y0_input')
    layer = Concatenate(name='concat')([layer, y0_input])

    # Post-flattening dense layers
    for i in range(len(FLATTENED_LAYERS)):
        layer = QDense(FLATTENED_LAYERS[i], kernel_quantizer=quantized_bits(WEIGHTS_BITS,INTEGER_BITS,alpha=1.0),
                    bias_quantizer=quantized_bits(BIAS_BITS,INTEGER_BITS,alpha=1.0), name=f'dense{i+1}')(layer)
        layer = QActivation(quantized_relu(10), name=f'relu{len(CONV_LAYER_DEPTHS)+i+1}')(layer)

    # Output layer (adjust based on your classification problem)
    output = QDense(3, kernel_quantizer=quantized_bits(WEIGHTS_BITS), bias_quantizer=quantized_bits(BIAS_BITS), name='dense_output')(layer)
    output = Activation("softmax", name='output_softmax')(output)
    # layer = QDense(1, kernel_quantizer=quantized_bits(WEIGHTS_BITS,INTEGER_BITS,alpha=1.0), 
    #                bias_quantizer=quantized_bits(BIAS_BITS,INTEGER_BITS,alpha=1.0), name='output_dense')(layer)
    # output = Activation("sigmoid", name='output_sigmoid')(layer)

    model = Model(inputs=[y_profile_input, y0_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy']) # loss='binary_crossentropy'

    return model


def qAEmodel(hyperparams):
    NUM_TIME_SLICES = hyperparams["NUM_TIME_SLICES"]
    AU_ENCODER_LAYERS = hyperparams["AU_ENCODER_LAYERS"]
    AU_DECODER_LAYERS = hyperparams["AU_DECODER_LAYERS"]
    CONV_LAYER_STRIDES = hyperparams["CONV_LAYER_STRIDES"]
    MAX_POOLING_SIZE = hyperparams["MAX_POOLING_SIZE"]
    WEIGHTS_BITS = hyperparams["WEIGHTS_BITS"]
    BIAS_BITS = hyperparams["BIAS_BITS"]
    INTEGER_BITS = hyperparams["INTEGER_BITS"]
    ACTIVATION_BITS = hyperparams["ACTIVATION_BITS"]
    LEARNING_RATE = hyperparams["LEARNING_RATE"]

    y_profile_input = Input(shape=(NUM_TIME_SLICES, 13, 1), name='y_profile_input')  # Adjust the shape based on your input
    layer = y_profile_input

    print(layer.shape)
    layer = Flatten(name='flatten_input')(layer)
    print(layer.shape)

    all_layer_specs = AU_ENCODER_LAYERS + AU_DECODER_LAYERS
    # Add encoder layers
    for i in range(len(all_layer_specs)):
        layer_specs = all_layer_specs[i]

        if isinstance(layer_specs, int):
            # Flatten layer if necessary
            if (len(layer.shape) != 2):
                layer = Flatten(name=f'flatten{i}')(layer)
            layer = QDense(layer_specs, kernel_quantizer=quantized_bits(WEIGHTS_BITS), bias_quantizer=quantized_bits(BIAS_BITS), name=f'dense{i+1}')(layer)
            layer = QActivation(quantized_relu(ACTIVATION_BITS))(layer)
            layer = BatchNormalization()(layer)

        elif isinstance(layer_specs, tuple):
            # Reshape layer if necessary
            if (len(layer.shape) != 4):
                layer = Reshape((NUM_TIME_SLICES, 13, 1), name=f'reshape{i}')(layer)
            layer = QConv2D(
            layer_specs[0],
            kernel_size=(layer_specs[1], layer_specs[2]),
            strides=CONV_LAYER_STRIDES,
            kernel_quantizer=quantized_bits(WEIGHTS_BITS,INTEGER_BITS,alpha=1.0),
            bias_quantizer=quantized_bits(BIAS_BITS,INTEGER_BITS,alpha=1.0),
            padding='same',
            use_bias=True,
            name=f'conv{i+1}'
            )(layer)
            layer = QActivation(quantized_relu(ACTIVATION_BITS), name=f'relu{i+1}')(layer)
            layer = MaxPooling2D(pool_size=MAX_POOLING_SIZE, name=f'maxpool{i+1}')(layer)
        else:
            raise ValueError(f"Unsupported layer specification: {layer_specs}")
        print(layer.shape)

    # Reshape output layer if necessary
    if (len(layer.shape) != 4):
        layer = Reshape((NUM_TIME_SLICES, 13, 1), name=f'reshape_output')(layer)

    ## TODO figure out what the output should be for autoencoder including y0 and supporting convolutions

    model = Model(inputs=y_profile_input, outputs=layer)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy']) # loss='binary_crossentropy'

    return model


# Pruning the model
def pruneFunction(layer, train_data_size, hyperparams):
    BATCH_SIZE = hyperparams["BATCH_SIZE"]
    FINAL_SPARSITY = hyperparams["FINAL_SPARSITY"]
    PRUNE_START_EPOCH = hyperparams["PRUNE_START_EPOCH"]
    NUM_PRUNE_EPOCHS = hyperparams["NUM_PRUNE_EPOCHS"]

    steps_per_epoch = train_data_size // BATCH_SIZE #input_train_data_combined.shape[0]
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.0,
            final_sparsity=FINAL_SPARSITY,
            begin_step=steps_per_epoch * PRUNE_START_EPOCH,
            end_step=steps_per_epoch * (PRUNE_START_EPOCH + NUM_PRUNE_EPOCHS),
            frequency=steps_per_epoch # prune after every epoch
        )
    }


    if isinstance(layer, QDense):
        if layer.name != 'output_softmax' and layer.name != 'dense2':
            print(f"pruning layer {layer.name}")
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        elif layer.name != 'output_softmax' and layer.name != 'dense1':
            print(f"pruning layer {layer.name}")
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        else:
            print(f"cannot prune layer {layer.name}")
            return layer

    else:
        print(f"cannot prune layer {layer.name}")
        return layer
    
def pruneFunctionWrapper(train_data_size, hyperparams):
    def wrapper(layer):
        return pruneFunction(layer, train_data_size, hyperparams)
    return wrapper
    

# Function to calculate sparsity
def calculate_sparsity(model):
    total_params = 0
    zero_params = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()[0]
            total_params += weights.size
            zero_params += np.sum(weights == 0)
    sparsity = zero_params / total_params
    return sparsity

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [5]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

def train_model(data, hyperparams):
    input_train_data_combined = data["input_train_data_combined"]
    target_train_data_coded = data["target_train_data_coded"]
    input_validate_data_combined = data["input_validate_data_combined"]
    target_validate_data_coded = data["target_validate_data_coded"]

    MODEL_TYPE = hyperparams["MODEL_TYPE"]
    PATIENCE = hyperparams["PATIENCE"]
    EPOCHS = hyperparams["EPOCHS"]
    BATCH_SIZE = hyperparams["BATCH_SIZE"]
    LEARNING_RATE = hyperparams["LEARNING_RATE"]
    POST_PRUNE_EPOCHS = hyperparams["POST_PRUNE_EPOCHS"]
    

    
    # Define the model
    if (MODEL_TYPE == "DNN"):
        model = qDNNmodel(hyperparams)
    elif (MODEL_TYPE == "CNN"):
        model = qCNNmodel(hyperparams)
    elif (MODEL_TYPE == "AE"):
        model = qAEmodel(hyperparams)
    else:
        raise ValueError("Not a supported model type")

    model.summary()
    print_qmodel_summary(model)
    print(f"Initial Sparsity: {calculate_sparsity(model) * 100:.2f}%")

    train_metrics = {}

    # Train the model
    earlyStop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=PATIENCE, restore_best_weights=True)
    if MODEL_TYPE == "DNN" or MODEL_TYPE == "CNN":
        history = model.fit(
            input_train_data_combined, target_train_data_coded,  # Training data and labels
            validation_data=(input_validate_data_combined, target_validate_data_coded),  # Validation data
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=[earlyStop_callback]
        )
    elif (MODEL_TYPE == "AE"):
        history = model.fit(
            input_train_data_combined, input_train_data_combined,  # Training data and labels
            validation_data=(input_validate_data_combined, input_validate_data_combined),  # Validation data
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=[earlyStop_callback]
        )
    else:
        raise ValueError("Not a supported model type")
    
    
    # Best at this step val_loss 0.7085
    train_metrics["val_loss"] = history.history['val_loss'][-1]

    # Prune the model
    model_pruned = keras.models.clone_model(model, clone_function=pruneFunctionWrapper(input_train_data_combined.shape[0], hyperparams))
    model_pruned.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])

    if MODEL_TYPE == "DNN" or MODEL_TYPE == "CNN":
        history = model_pruned.fit(
            input_train_data_combined, target_train_data_coded,
            validation_data=(input_validate_data_combined, target_validate_data_coded),
            epochs=POST_PRUNE_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks = [pruning_callbacks.UpdatePruningStep()]
        )
    if MODEL_TYPE == "AE":
        history = model_pruned.fit(
            input_train_data_combined, input_train_data_combined,
            validation_data=(input_validate_data_combined, input_validate_data_combined), 
            epochs=POST_PRUNE_EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks = [pruning_callbacks.UpdatePruningStep()]
        ) 

    model = strip_pruning(model_pruned)
    # train_metrics["pruned_sparsity"] = calculate_sparsity(model)

    try:
        train_metrics["pruned_val_loss"] = history.history['val_loss'][-1]
    except:
        print("Error: no post-pruning val_loss found")
        train_metrics["pruned_val_loss"] = train_metrics["val_loss"]

    return model, train_metrics


######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [6]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

def test_model(data, model):
    input_test_data_combined = data["input_test_data_combined"]
    target_test_data_coded = data["target_test_data_coded"]

    
    # Test the model at threshold 0.5
    predictions_prob = model.predict(input_test_data_combined)[:,0]
    predictions_labels = (predictions_prob >= 0.5).astype(int).flatten()

    # Test the model at different thresholds
    thresholds = np.linspace(0.0, 1.0, 1000)
    signal_efficiencies = []
    background_rejections = []
    max_sum_se = 0
    max_sum_br = 0

    for threshold in thresholds:
        # predicted_class = ((predictions_prob[:, 0] + threshold > predictions_prob[:, 1]) & (predictions_prob[:, 0] + threshold > predictions_prob[:, 2])).astype(int)
        predicted_class = (predictions_prob > threshold).astype(int)
        # Compute confusion matrix
        cm = confusion_matrix(target_test_data_coded[:, 0], predicted_class)

        # Calculate signal efficiency and background rejection
        signal_efficiency = cm[1, 1] / np.sum(cm[1, :])
        background_rejection = cm[0, 0] / np.sum(cm[0, :])

        # Store metrics
        signal_efficiencies.append(signal_efficiency)
        background_rejections.append(background_rejection)

        # get maximum added score
        if signal_efficiency + background_rejection > max_sum_se + max_sum_br:
            max_sum_se = signal_efficiency
            max_sum_br = background_rejection
    
    test_results = {
        "predictions_prob": predictions_prob,
        "predictions_labels": predictions_labels,
        "thresholds": thresholds,
        "signal_efficiencies": signal_efficiencies,
        "background_rejections": background_rejections,
        "max_sum_se": max_sum_se,
        "max_sum_br": max_sum_br,
    }

    return test_results

def ShowConfusionMatrix(test_results):
    target_test_data_coded = test_results["target_test_data_coded"]
    predictions_labels = test_results["predictions_labels"]

    cm = confusion_matrix(target_test_data_coded[:,0], predictions_labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='YlGnBu')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

def showMetricsByThreshold(test_results):
    thresholds = test_results["thresholds"]
    signal_efficiencies = test_results["signal_efficiencies"]
    background_rejections = test_results["background_rejections"]

    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, signal_efficiencies, label='Signal Efficiency')
    plt.plot(thresholds, background_rejections, label='Background Rejection')
    plt.xlabel('Threshold')
    plt.ylabel('Metric Value')
    plt.title('Effect of Threshold on Signal Efficiency and Background Rejection')
    plt.legend()
    plt.grid(True)
    plt.show()

def showEfficiencyVSRejection(test_results):
    signal_efficiencies = test_results["signal_efficiencies"]
    background_rejections = test_results["background_rejections"]

    plt.figure(figsize=(10, 6))
    plt.plot(signal_efficiencies, background_rejections, marker='o')
    plt.xlabel('Signal Efficiency')
    plt.ylabel('Background Rejection')
    plt.title('Background Rejection vs. Signal Efficiency')
    plt.grid(True)
    plt.show()

def find_closest(sorted_array, value):
    # Ensure the array is a NumPy array
    sorted_array = np.array(sorted_array)
    # Compute the absolute difference
    abs_diff = np.abs(sorted_array - value)
    # Find the index of the minimum difference
    closest_index = np.argmin(abs_diff)
    return closest_index

def getTargetMetrics(test_results):
    signal_efficiencies = test_results["signal_efficiencies"]
    background_rejections = test_results["background_rejections"]

    target_efficiencies = [0.90, 0.93, 0.96, 0.98, 0.99, 0.995, 0.999]
    metrics = []
    for target in target_efficiencies:
        index = find_closest(signal_efficiencies, target)
        metrics.append((signal_efficiencies[index], background_rejections[index]))
        # print(f"Signal Efficiency: {signal_efficiencies[index]*100:.1f}%,",f"Background Rejections: {background_rejections[index]*100:.1f}%")
    return metrics

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [7]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

def hyperparameter_search(data, base_hyperparams, param_grid, result_file='hyperparameter_results.json'):

    # Load existing results from file if it exists
    if os.path.exists(result_file):
        with open(result_file, 'r') as file:
            all_results = json.load(file)
    else:
        all_results = {}

    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        hyperparams = dict(zip(keys, v))
        # Update base hyperparameters with the current set
        current_hyperparams = base_hyperparams.copy()
        current_hyperparams.update(hyperparams)
        MODEL_TYPE = current_hyperparams["MODEL_TYPE"]

        # Convert hyperparameters to a string for use as a dictionary key
        hyperparams_str = json.dumps(current_hyperparams, sort_keys=True)

        # Check if these hyperparameters have been tried before
        if hyperparams_str in all_results:
            print(f"Skipping already tested hyperparameters: {current_hyperparams}")
            continue

        print(f"Testing hyperparameters: {current_hyperparams}")

        # Train the model
        model, train_metrics = train_model(data, current_hyperparams)

        if (MODEL_TYPE == "DNN" or MODEL_TYPE == "CNN"):
            # Test the model
            test_results = test_model(data, model)
            metrics = getTargetMetrics(test_results)
            metrics_str = ", ".join([f"({m1:.4f}, {m2:.4f})" for m1, m2 in metrics])
            
            test_scores = {
                "max_sum_se": test_results["max_sum_se"],
                "max_sum_br": test_results["max_sum_br"],
                "metrics": metrics_str,
            }
            # Add all keys and values from train_metrics into test_scores
            test_scores.update(train_metrics)
        else:
            test_scores = train_metrics

        # Save the results to the file
        all_results[hyperparams_str] = test_scores
        with open(result_file, 'w') as file:
            json.dump(all_results, file, indent=4)


        # If new best found, email alex
        if test_scores["pruned_val_loss"] < find_min_pruned_val_loss(result_file):

            # email results
            model_name = "undefined"
            if (current_hyperparams["MODEL_TYPE"] == "DNN"):
                model_name = "Dean"
            elif (current_hyperparams["MODEL_TYPE"] == "CNN"):
                model_name = "Connor"
            elif (current_hyperparams["MODEL_TYPE"] == "AE"):
                model_name = "Audrey"
            send_email_notification("ML Training Report", 
                f' \
                Your model {model_name} has finished training. \
                He got a grade of {test_scores["max_sum_se"]*100:.1f}% in SE and {test_scores["max_sum_br"]*100:.1f}% in BR. \
                New lowest validated loss: {test_scores["pruned_val_loss"]} \n \
                All metrics: {metrics_str} \n \
                Hyperparams: {hyperparams_str} \
                ')

    return all_results


def find_min_pruned_val_loss(result_file='hyperparameter_results.json'):
    # Load existing results from file
    if os.path.exists(result_file):
        with open(result_file, 'r') as file:
            all_results = json.load(file)
    else:
        print(f"No results found in {result_file}")
        return None

    min_loss = float('inf')
    min_hyperparams = None

    # Iterate through the results to find the minimum pruned_val_loss
    for hyperparams_str, results in all_results.items():
        if "pruned_val_loss" in results:
            pruned_val_loss = results["pruned_val_loss"]
            if pruned_val_loss < min_loss:
                min_loss = pruned_val_loss
                min_hyperparams = hyperparams_str

    # Print the hyperparameters with the minimum pruned_val_loss
    if min_hyperparams is not None:
        print(f"Hyperparameters with minimum pruned_val_loss: {min_hyperparams}")
        print(f"Minimum pruned_val_loss: {min_loss}")
    else:
        print("No entry with pruned_val_loss found")

    return min_loss

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

In [14]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

HYPERPARAMETERS = {
    # Model Type
    "MODEL_TYPE": "AE",  # DNN or CNN or AE
    # Input format
    "NUM_TIME_SLICES": 8,
    "TRAIN_PT_THRESHOLD": 2,  # in GeV
    "TEST_PT_THRESHOLD": 2,  # in GeV
    "INPUT_SCALING": "MinMax", # Standard, MinMax or Log
    # DNN model mormat
    "DNN_LAYERS": [32, 16, 8], # [32, (16, 3, 3), (16, 3, 3), 32]
    # CNN model format
    "CONV_LAYER_DEPTHS": [4, 7],
    "CONV_LAYER_KERNELS": [(3, 3), (3, 3)],
    "FLATTENED_LAYERS": [7],
    "CONV_LAYER_STRIDES": (1, 1),
    "MAX_POOLING_SIZE": (2, 2),
    # AU model format. n is dense layer of size n, (n, m, l) is conv layer of dept n and kernel size (m, l)
    ## TODO: add maxpooling and sampling and crop to support CNN for AE
    "AU_ENCODER_LAYERS": [64],
    "AU_DECODER_LAYERS": [8*13], # Includes output. Must result in NUM_TIME_SLICES * 13 or (NUM_TIME_SLICES, 13) sized final layer
    # Model quantization
    "WEIGHTS_BITS": 10,
    "BIAS_BITS": 10,
    "ACTIVATION_BITS": 15,
    "INTEGER_BITS": 2,
    # Training
    "LEARNING_RATE": 0.001,
    "BATCH_SIZE": 1024,  # Number of samples per gradient update
    "EPOCHS": 100,  # Number of epochs to train
    "PATIENCE": 20,  # Stop after this number of epochs without improvement
    # Pruning
    "PRUNE_START_EPOCH": 0,  # Number of epochs before pruning
    "NUM_PRUNE_EPOCHS": 10,
    "FINAL_SPARSITY": 0.35,
    "POST_PRUNE_EPOCHS": 50,
}

SAVE_FILE = "small"+HYPERPARAMETERS["MODEL_TYPE"]+"_results.json"

param_grid = {
    "AU_ENCODER_LAYERS": [[32], [64, 32], [96, 32]],
    "AU_DECODER_LAYERS": [[8*13], [64, 8*13]],
    "FINAL_SPARSITY": [0.35, 0.4, 0.45],
}

find_min_pruned_val_loss(result_file=SAVE_FILE)

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

Hyperparameters with minimum pruned_val_loss: {"ACTIVATION_BITS": 15, "AU_DECODER_LAYERS": [64, 104], "AU_ENCODER_LAYERS": [32], "BATCH_SIZE": 1024, "BIAS_BITS": 10, "CONV_LAYER_DEPTHS": [4, 7], "CONV_LAYER_KERNELS": [[3, 3], [3, 3]], "CONV_LAYER_STRIDES": [1, 1], "DNN_LAYERS": [128, 64, 32, 16], "EPOCHS": 100, "FINAL_SPARSITY": 0.45, "FLATTENED_LAYERS": [7], "INPUT_SCALING": "MinMax", "INTEGER_BITS": 2, "LEARNING_RATE": 0.001, "MAX_POOLING_SIZE": [2, 2], "MODEL_TYPE": "AE", "NUM_PRUNE_EPOCHS": 10, "NUM_TIME_SLICES": 8, "PATIENCE": 20, "POST_PRUNE_EPOCHS": 50, "PRUNE_START_EPOCH": 0, "TEST_PT_THRESHOLD": 2, "TRAIN_PT_THRESHOLD": 2, "WEIGHTS_BITS": 10}
Minimum pruned_val_loss: 0.34005746245384216


0.34005746245384216

In [10]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

input_data, target_data = load_dataset()
data = process_dataset(input_data, target_data, HYPERPARAMETERS) 
# Depends only on: NUM_TIME_SLICES MODEL_TYPE TRAIN_PT_THRESHOLD TEST_PT_THRESHOLD

######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

Total memory: 376.23 GB
Available memory: 275.21 GB
Used memory: 90.24 GB
Memory usage percentage: 26.9%
Total memory: 376.23 GB
Available memory: 263.59 GB
Used memory: 101.85 GB
Memory usage percentage: 29.9%


In [11]:
model, train_metrics = train_model(data, HYPERPARAMETERS)

(None, 8, 13, 1)
(None, 104)
(None, 64)
(None, 104)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 y_profile_input (InputLaye  [(None, 8, 13, 1)]        0         
 r)                                                              
                                                                 
 flatten_input (Flatten)     (None, 104)               0         
                                                                 
 dense1 (QDense)             (None, 64)                6720      
                                                                 
 q_activation (QActivation)  (None, 64)                0         
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense2 (

In [12]:
model.layers

In [ ]:
dense_layer = model.layers[2]
bn_layer = model.layers[2]

W, b = dense_layer.get_weights()
gamma, beta, moving_mean, moving_var = bn_layer.get_weights()

# Calculate new weights and biases
epsilon = bn_layer.epsilon
std = np.sqrt(moving_var + epsilon)
new_W = gamma / std * W
new_b = gamma / std * (b - moving_mean) + beta

# Create new model without BN layer
inputs = Input(shape=(64,))
x = Dense(32, weights=[new_W, new_b])(inputs)
outputs = Dense(10)(x)

new_model = Model(inputs, outputs)

# Verify the new model
new_model.summary()

In [91]:
model.save(f'./{HYPERPARAMETERS["MODEL_TYPE"]}_best_perfomance.h5')

/sdf/home/a/alexyue/miniconda3/envs/SmartPixel/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

# results = hyperparameter_search(data, HYPERPARAMETERS, param_grid, result_file=SAVE_FILE)

try:
    results = hyperparameter_search(data, HYPERPARAMETERS, param_grid, result_file=SAVE_FILE)
    send_email_notification("All done with hyperparameter search", 'Done!')
except Exception as e:
    print("Error encountered:", e)
    send_email_notification("Hyperparameter search ran into an error", 'Go fix it')
######################################################################################################
##################################         TESTING          ##########################################
######################################################################################################

Testing hyperparameters: {'MODEL_TYPE': 'AE', 'NUM_TIME_SLICES': 8, 'TRAIN_PT_THRESHOLD': 2, 'TEST_PT_THRESHOLD': 2, 'INPUT_SCALING': 'MinMax', 'DNN_LAYERS': [128, 64, 32, 16], 'CONV_LAYER_DEPTHS': [4, 7], 'CONV_LAYER_KERNELS': [(3, 3), (3, 3)], 'FLATTENED_LAYERS': [7], 'CONV_LAYER_STRIDES': (1, 1), 'MAX_POOLING_SIZE': (2, 2), 'AU_ENCODER_LAYERS': [32], 'AU_DECODER_LAYERS': [104], 'WEIGHTS_BITS': 10, 'BIAS_BITS': 10, 'ACTIVATION_BITS': 15, 'INTEGER_BITS': 2, 'LEARNING_RATE': 0.001, 'BATCH_SIZE': 1024, 'EPOCHS': 100, 'PATIENCE': 20, 'PRUNE_START_EPOCH': 0, 'NUM_PRUNE_EPOCHS': 10, 'FINAL_SPARSITY': 0.35, 'POST_PRUNE_EPOCHS': 50}
(None, 8, 13, 1)
(None, 104)
(None, 32)
(None, 104)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 y_profile_input (InputLaye  [(None, 8, 13, 1)]        0         
 r)                                                              
                                

In [2]:
def format_metrics(metrics):
    # Convert metrics list of tuples to a formatted string
    return ", ".join([f"({m1:.4f}, {m2:.4f})" for m1, m2 in metrics])

def reformat_hyperparameter_results(input_file, output_file):
    # Read the original JSON file
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Process and format the metrics
    for key, value in data.items():
        if "metrics" in value:
            value["metrics"] = format_metrics(value["metrics"])

    # Write the updated data to the new JSON file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

# Define input and output file names
input_file = 'OLD_CNN_hyperparameter_results.json'
output_file = 'CNN_hyperparameter_results.json'

# Call the function to reformat the JSON data
reformat_hyperparameter_results(input_file, output_file)